## DataFrames creation, light EDA & some data cleaning

In [22]:
import pandas as pd

# df_http
df_http = pd.read_csv('./data/input/polling.csv', index_col = 0)
print(df_http.shape)
print(df_http.info())
df_http.head()

(350445, 4)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 350445 entries, 0 to 350444
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   creation_time  350445 non-null  object
 1   device_id      350445 non-null  object
 2   error_code     1326 non-null    object
 3   status_code    350445 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 13.4+ MB
None


,creation_time,device_id,error_code,status_code
0,2020-02-26 19:16:40.481,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
1,2020-02-26 19:16:25.035,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
2,2020-02-26 18:31:52.126,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
3,2020-02-26 18:31:17.043,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200
4,2020-02-26 18:31:01.738,d0460656-95e0-4844-b515-36ef46c2d620,NaN,200


In [12]:
# df_status
df_status = pd.read_csv('./data/input/connectivity_status.csv', index_col = 0)
print(df_status.shape)
df_status.head()

(35559, 3)


,creation_time,status,device_id
0,2020-02-26 19:31:29.998,OFFLINE,00083c70-7f54-4324-94bd-b0cb0be78baf
1,2020-02-26 19:31:39.677,ONLINE,00083c70-7f54-4324-94bd-b0cb0be78baf
2,2020-02-26 20:08:42.789,OFFLINE,000fec74-a5b4-40fc-b93c-eef3b2afb87e
3,2020-02-26 20:09:03.448,ONLINE,000fec74-a5b4-40fc-b93c-eef3b2afb87e
4,2020-02-26 00:30:03.439,OFFLINE,001d3c67-99fd-43b7-8207-f420fa303a49


In [14]:
# df_orders
df_orders = pd.read_csv('./data/input/orders.csv', index_col = 0)

print(f"Orders count pre-cleaning: {df_orders.shape[0]}")
# we should get rid of rows with no device_id associated since we're focusing on metrics related to dispatched orders only
print(f"Orders count not dispatched to a device: {len(df_orders.loc[df_orders['device_id'].isna(), 'order_id'].unique())}")

df_orders = df_orders.loc[df_orders.device_id.notna()]
print(f"Orders count post-cleaning: {df_orders.shape[0]}")
df_orders.head()

Orders count pre-cleaning: 2357
Orders count not dispatched to a device: 26
Orders count post-cleaning: 2331


,order_creation_time,order_id,device_id
5,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862
6,2020-02-26 10:34:36,102492697,2bb11f99-ab21-4628-abe6-b919da8fbf34
7,2020-02-26 11:32:47,102500373,2aec0e20-e1d8-4323-9b12-f066856488a7
8,2020-02-26 11:46:17,102503173,37638585-a181-4265-aeb2-d9e284bb30c3
9,2020-02-26 11:38:40,102501909,53d16c33-5980-4ad8-9032-f85d686d2855


----

## Data manipulation

In [27]:
import random

df = pd.merge(df_orders, df_http, left_on = 'device_id', right_on = 'device_id', how = 'left', indicator = True)
display(df.head())

print(f"Orders count dispatched to unknown devices: {df.groupby('_merge').size()['left_only']}") # associated device_id is missing from the polling dataset
inc_orders = list(df.loc[df._merge == 'left_only', 'order_id'].unique()) # here's the dataset (as a list) of inconsistent orders for further analysis
print(f"Here's one of those oders to double check its metrics later on down the pipeline: {inc_orders[random.randint(0, len(inc_orders))]}")

,order_creation_time,order_id,device_id,creation_time,error_code,status_code,_merge
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:28:41.657,NaN,200.0,both
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:51.879,NaN,200.0,both
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:35.925,NaN,200.0,both
4,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:26:34.578,NaN,200.0,both


Orders count dispatched to unknown devices: 71
Here's one of those oders to double check its metrics later on down the pipeline: 102639577


In [28]:
df[['order_creation_time', 'creation_time']] = df[['order_creation_time', 'creation_time']].apply(pd.to_datetime) # convert time columns to DateTime type
print(df.shape)
df.head()

(4710527, 7)


,order_creation_time,order_id,device_id,creation_time,error_code,status_code,_merge
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:29:34.363,NaN,200.0,both
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:28:41.657,NaN,200.0,both
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:51.879,NaN,200.0,both
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:27:35.925,NaN,200.0,both
4,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 05:26:34.578,NaN,200.0,both


In [29]:
# df.set_index('order_id', inplace = True)
# df.head()

In [31]:
def cut_df(
    df: pd.DataFrame,
    m_os: int
) -> pd.DataFrame:
    """Slice a DataFrame over a specific portion of time.

    Args:
        df (pd.DataFrame): a DataFrame to be sliced.
        m_os (int): a time offset (minutes).
    
    Returns:
        df_cut (pd.DataFrame): the slice of the DataFrame whose rows
            are comprised between the order creation time and:
                - the preceding time segment determined by the offset (if m_os < 0)
                - the following time segment determined by the offset (if m_os > 0)
    
    Raises:
        ValueError: `m_os` param has to be either positive
            or negative, not a zero time segment (0).
    """
    if m_os < 0:
        df_cut = df.loc[df['creation_time'].between(df['order_creation_time'] + pd.DateOffset(minutes = m_os), df['order_creation_time'])]
        df_cut['timeslice'] = f"{abs(m_os)}m_before"

    elif m_os > 0:
        df_cut = df.loc[df['creation_time'].between(df['order_creation_time'], df['order_creation_time'] + pd.DateOffset(minutes = m_os))]
        df_cut['timeslice'] = f"{abs(m_os)}m_after"

    else:
        raise ValueError('`m_os` param cannot be 0')
    return df_cut

In [39]:
pd.options.mode.chained_assignment = None # this prevents the good old SettingWithCopyWarning to jump out in the cell output

slices = [-3, -60, 3]

df_list = [cut_df(df, i) for i in slices]
df_raw = pd.concat(df_list, axis = 0, ignore_index = True) # slices union (row-wise concat)

print(df_raw.shape)
# print(df_raw.groupby('timeslice').size())
df_raw.loc[df_raw.order_id == 102452116].head()

(354527, 8)


,order_creation_time,order_id,device_id,creation_time,error_code,status_code,_merge,timeslice
0,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:08:12.125,ECONNABORTED,0.0,both,3m_before
1,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:06:46.961,NaN,200.0,both,3m_before
2,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:06:20.304,NaN,200.0,both,3m_before
3,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:06:18.986,NaN,200.0,both,3m_before
16479,2020-02-26 00:08:19,102452116,d0bc996f-72d2-4ec2-8f40-d82d81120862,2020-02-26 00:08:12.125,ECONNABORTED,0.0,both,60m_before


In [37]:
# dropna = False is required since groupby() is by default filtering out those groups whose aggregating columns are NaN
df_report = df_raw.groupby(['order_id', 'timeslice', 'status_code', 'error_code'], dropna = False).size().reset_index(name = 'rows_count')
df_report.head()
# resulting df now contains counts for all the possible combinations of our 4 grouping features

,order_id,timeslice,status_code,error_code,rows_count
0,102452116,3m_after,200.0,NaN,1
1,102452116,3m_before,0.0,ECONNABORTED,1
2,102452116,3m_before,200.0,NaN,3
3,102452116,60m_before,0.0,ECONNABORTED,1
4,102452116,60m_before,200.0,NaN,31


In [106]:
# df_report = df_report.set_index('order_id') # setting order_id as the new index
# df_report.head()

### Computing closest polling timestamps

In [40]:
# immediately preceding and following order creation times' events have to be computed across an unbounded period of time, that's why I'm using the first merged dataset: 'df'

# sorting here is somehow mandatory since later I'll use head(1) to pick up the two closest rows (before and after) to my time pivot point ('order_creation_time')
df_clst = df[['order_id', 'order_creation_time', 'creation_time']].sort_values(['order_id', 'creation_time'], ascending = [True, False])
df_clst['abs_dt_diff'] = abs(df['order_creation_time'] - df['creation_time'])
df_clst['poll_before_order'] = (df['order_creation_time'] > df['creation_time'])

df_clst.head()

,order_id,order_creation_time,creation_time,abs_dt_diff,poll_before_order
0,102452116,2020-02-26 00:08:19,2020-02-26 05:29:34.363,0 days 05:21:15.363000,False
1,102452116,2020-02-26 00:08:19,2020-02-26 05:28:41.657,0 days 05:20:22.657000,False
2,102452116,2020-02-26 00:08:19,2020-02-26 05:27:51.879,0 days 05:19:32.879000,False
3,102452116,2020-02-26 00:08:19,2020-02-26 05:27:35.925,0 days 05:19:16.925000,False
4,102452116,2020-02-26 00:08:19,2020-02-26 05:26:34.578,0 days 05:18:15.578000,False


In [41]:
df_clst_sg = df_clst.groupby(['order_id', 'poll_before_order']).head(1)
df_clst_sg.head()

,order_id,order_creation_time,creation_time,abs_dt_diff,poll_before_order
0,102452116,2020-02-26 00:08:19,2020-02-26 05:29:34.363,0 days 05:21:15.363000,False
751,102452116,2020-02-26 00:08:19,2020-02-26 00:08:12.125,0 days 00:00:06.875000,True
3332698,102452190,2020-02-26 00:08:41,2020-02-26 05:29:34.363,0 days 05:20:53.363000,False
3333449,102452190,2020-02-26 00:08:41,2020-02-26 00:08:12.125,0 days 00:00:28.875000,True
4101120,102453036,2020-02-26 00:13:57,2020-02-26 05:29:34.363,0 days 05:15:37.363000,False


In [42]:
df_clst_fl = df_clst_sg.pivot_table(index = ['order_id'], columns = 'poll_before_order', values = 'creation_time', aggfunc = max)
df_clst_fl.columns = ['after_poll_ts', 'before_poll_ts']

df_clst_fl.head()

,after_poll_ts,before_poll_ts
order_id,,
102452116,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
102452190,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
102453036,2020-02-26 05:29:34.363,2020-02-26 00:13:12.644
102453649,2020-02-26 05:29:34.363,2020-02-26 00:17:29.623
102453774,2020-02-26 05:29:34.363,2020-02-26 00:18:06.815


In [52]:
# df_rep = pd.concat([df_report, df_clst_fl], axis = 1)
# df_rep.head()

In [55]:
# adding the 2 new computed columns to my report with a inner join on the order_id common field -> this merge should be done at the end :D
df_report_sa = pd.merge(df_report, df_clst_fl, left_on = 'order_id', right_on = 'order_id', how = 'inner')
df_report_sa.head()

,order_id,timeslice,status_code,error_code,rows_count,after_poll_ts,before_poll_ts
0,102452116,3m_after,200.0,NaN,1,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
1,102452116,3m_before,0.0,ECONNABORTED,1,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
2,102452116,3m_before,200.0,NaN,3,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
3,102452116,60m_before,0.0,ECONNABORTED,1,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
4,102452116,60m_before,200.0,NaN,31,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125


In [57]:
import numpy as np

# computing status_code KPIs
df_pv_sc = df_report.pivot_table(index = ['order_id', 'timeslice'], columns = 'status_code', values = 'rows_count', aggfunc = np.sum)#, fill_value = 0)
df_pv_sc['polling_events'] = df_pv_sc.loc[:, list(df_pv_sc.columns)].sum(axis = 1) # creating the polling events count column as the sum of all the other columns
df_pv_sc.head()

status_code           0.0  200.0  401.0  polling_events
order_id  timeslice                                    
102452116 3m_after    NaN    1.0    NaN             1.0
          3m_before   1.0    3.0    NaN             4.0
          60m_before  1.0   31.0    NaN            32.0
102452190 3m_after    NaN    2.0    NaN             2.0
          3m_before   1.0    3.0    NaN             4.0

In [58]:
# computing error_code KPIs
df_pv_ec = df_report.pivot_table(index = ['order_id', 'timeslice'], columns = 'error_code', values = 'rows_count', aggfunc = np.sum)#, fill_value = 0)
# we don't need to compute count of responses without error codes. since it has to be equal to 200 responses count
df_pv_ec.head()

error_code            ECONNABORTED  GENERIC_ERROR
order_id  timeslice                              
102452116 3m_before            1.0            NaN
          60m_before           1.0            NaN
102452190 3m_before            1.0            NaN
          60m_before           1.0            NaN
102453036 60m_before           1.0            NaN

In [59]:
df_final = pd.concat([df_pv_sc, df_pv_ec], axis = 1)
df_final = df_final.fillna(0).astype(int)
df_final.head()

0.0  200.0  401.0  polling_events  ECONNABORTED  \
order_id  timeslice                                                     
102452116 3m_after      0      1      0               1             0   
          3m_before     1      3      0               4             1   
          60m_before    1     31      0              32             1   
102452190 3m_after      0      2      0               2             0   
          3m_before     1      3      0               4             1   

                      GENERIC_ERROR  
order_id  timeslice                  
102452116 3m_after                0  
          3m_before               0  
          60m_before              0  
102452190 3m_after                0  
          3m_before               0

In [69]:
df_report_csv = df_final.join(df_clst_fl, how = 'inner') # YEEEEEEEEAH
df_report_csv.head()

0.0  200.0  401.0  polling_events  ECONNABORTED  \
order_id  timeslice                                                     
102452116 3m_after      0      1      0               1             0   
          3m_before     1      3      0               4             1   
          60m_before    1     31      0              32             1   
102452190 3m_after      0      2      0               2             0   
          3m_before     1      3      0               4             1   

                      GENERIC_ERROR           after_poll_ts  \
order_id  timeslice                                           
102452116 3m_after                0 2020-02-26 05:29:34.363   
          3m_before               0 2020-02-26 05:29:34.363   
          60m_before              0 2020-02-26 05:29:34.363   
102452190 3m_after                0 2020-02-26 05:29:34.363   
          3m_before               0 2020-02-26 05:29:34.363   

                              before_poll_ts  
order_id  timeslice                           
102452116 3m_after   2020-02-26 00:08:12.125  
          3m_before  2020-02-26 00:08:12.125  
          60m_before 2020-02-26 00:08:12.125  
102452190 3m_after   2020-02-26 00:08:12.125  
          3m_before  2020-02-26 00:08:12.125

### CSV

In [60]:
# df_report = df.groupby(['order_id', 'status_code', 'error_code'], dropna = False).size().reset_index(name = 'counts')
# groupby by default is excluding those groups whose grouping columns are NaN (in this specific case successful polling events, obviously not associated with an error code)
# df_report.head()
df_final.to_csv('./data/output/report.csv') # this csv is honestly good enough

# HERE

In [61]:
df_shift = df[['order_id', 'order_creation_time', 'creation_time']].sort_values(['order_id', 'creation_time'], ascending = [True, False])
# testa['abs_dt_diff'] = abs(df['creation_time'] - df['order_creation_time'])
df_shift['abs_dt_diff'] = abs(df['order_creation_time'] - df['creation_time'])
df_shift['poll_before_order'] = (df['order_creation_time'] > df['creation_time'])

# print(testa.head())
# testa.groupby('order_id').min('order_pollingevent_diff')
df_shift.head()

,order_id,order_creation_time,creation_time,abs_dt_diff,poll_before_order
0,102452116,2020-02-26 00:08:19,2020-02-26 05:29:34.363,0 days 05:21:15.363000,False
1,102452116,2020-02-26 00:08:19,2020-02-26 05:28:41.657,0 days 05:20:22.657000,False
2,102452116,2020-02-26 00:08:19,2020-02-26 05:27:51.879,0 days 05:19:32.879000,False
3,102452116,2020-02-26 00:08:19,2020-02-26 05:27:35.925,0 days 05:19:16.925000,False
4,102452116,2020-02-26 00:08:19,2020-02-26 05:26:34.578,0 days 05:18:15.578000,False


In [62]:
# testa.groupby('order_id').size().reset_index(name = 'count').sort_values('count').query('count > 10 and count < 20')
# testa.shape

In [63]:
# testa.loc[testa.order_id == 102632539].sort_values('abs_dt_diff')

# poll_before_order -> True => polling event before order -> head(1)
# poll_before_order -> False => polling event after order -> head(1)

# testa.loc[testa.order_id == 102622694].sort_values('abs_dt_diff')

In [65]:
df_shift_stage = df_shift.groupby(['order_id', 'poll_before_order']).head(1)
df_shift_stage.head()

,order_id,order_creation_time,creation_time,abs_dt_diff,poll_before_order
0,102452116,2020-02-26 00:08:19,2020-02-26 05:29:34.363,0 days 05:21:15.363000,False
751,102452116,2020-02-26 00:08:19,2020-02-26 00:08:12.125,0 days 00:00:06.875000,True
3332698,102452190,2020-02-26 00:08:41,2020-02-26 05:29:34.363,0 days 05:20:53.363000,False
3333449,102452190,2020-02-26 00:08:41,2020-02-26 00:08:12.125,0 days 00:00:28.875000,True
4101120,102453036,2020-02-26 00:13:57,2020-02-26 05:29:34.363,0 days 05:15:37.363000,False


In [75]:
df_shift_final = df_shift_stage.pivot_table(index = ['order_id'], columns = 'poll_before_order', values = 'creation_time', aggfunc = max)
df_shift_final.columns = ['after_poll_ts', 'before_poll_ts']

df_shift_final.head()

# False -> after_poll
# True -> before_poll

,after_poll_ts,before_poll_ts
order_id,,
102452116,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
102452190,2020-02-26 05:29:34.363,2020-02-26 00:08:12.125
102453036,2020-02-26 05:29:34.363,2020-02-26 00:13:12.644
102453649,2020-02-26 05:29:34.363,2020-02-26 00:17:29.623
102453774,2020-02-26 05:29:34.363,2020-02-26 00:18:06.815


# Need to concat df_shift_final to the report before the second one become multiindex!

In [76]:
df_final_2 = pd.concat([df_final, df_shift_final], axis = 1)
df_final_2.head()


NotImplementedError: Can only union MultiIndex with MultiIndex or Index of tuples, try mi.to_flat_index().union(other) instead.